# RSS: Economista

## 1. Import Libraries

In [2]:
import feedparser
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import datetime
from termcolor import colored
from IPython.display import display, Markdown, Latex, HTML
import re
from IPython.display import Image
import urllib

In [3]:
def get_only_text1(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    # Eliminar Última Hora
    rest = soup.find_all('p', attrs={'class' : "line-clamp_x2 ue-c-widget__article-headline"})[0].text
    index = text.find(rest)
    final_text = text[:index]
    return [final_text,soup.title.text]

def get_only_text(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    return [text,soup.title.text]

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [4]:
def economista_news(economista_entries,word_count=100):
    economista = {}
    for i in range(len(economista_entries)):
        noticia = {'newspaper':None,'title':None,'headline':None,'summarise':None,'date':None,'link':None,'text':None,'current_date':None,'img':None,'premium':None}
        # Periodico
        noticia['newspaper'] = find_between(s=economista_entries[i]['id'], first='www.el', last='.' )
        # Title
        noticia['title'] = economista_entries[i]['title']
        # Headline
        headline =  re.sub(pattern='&quot;',repl='"',string=economista_entries[i]['summary_detail']['value'])
        noticia['headline'] = headline
        # Date
        noticia['date'] = {'date1':economista_entries[i]['published'],'date2':economista_entries[i]['published_parsed']}
        # Link
        noticia['link'] = economista_entries[i]['id']
        # Text
        text = get_only_text(economista_entries[i]['id'])[0][len(headline)+1:]
        noticia['text'] = text
        # Summarise
        noticia['summarise'] = summarize(text, word_count=word_count)
        # Current Date
        noticia['current_date'] = datetime.datetime.now().timetuple()
        # Image
        noticia['img'] = None
        economista.update({'Noticia'+'_'+str("%02d")%(i+1):noticia})
    return economista

## 2. RSS Economista

In [5]:
url_economista = 'https://www.eleconomista.es/rss/rss-empresas.php'
economista = feedparser.parse(url_economista)
economista_entries = economista['entries']

print('Total Entradas Economista:',len(economista_entries))

Total Entradas Economista: 1


In [6]:
economista = economista_news(economista_entries,word_count=100)

In [7]:
yesterday = datetime.datetime.today() - datetime.timedelta(days=1)
before_yes = datetime.datetime.today() - datetime.timedelta(days=2)

for i in list(economista.keys()):
    if economista[i]['date']['date2'][2] in [datetime.datetime.today().day,yesterday.day,before_yes.day]:
        print(colored((economista[i]['newspaper']).upper(),'green',attrs=['bold']))
        print(colored(economista[i]['date']['date1'],'grey',attrs=['bold']))
        print(colored(economista[i]['title'],'magenta',attrs=['bold']))
        print(colored('Headline','blue'))
        print(economista[i]['headline'])
        print(colored('Summarise','blue'))
        
        if economista[i]['summarise'] != 'Premium Content':
            print(economista[i]['summarise'])
        else:
            print(colored(economista[i]['summarise'],'red'))
        print(colored('Text','blue'))
        print(economista[i]['text'])
        display(Markdown('<a href="'+economista[i]['link']+'">Link</a>'))
        print('\n')

ECONOMISTA
Fri, 10 Jul 2020 11:08:13 +0200
Iberdrola encarga a Ingeteam su mayor batería de ion litio en el mundo
Headline
Iberdrola ha adjudicado a Ingeteam el suministro de su mayor sistema de almacenamiento de energía con baterías de ion litio. El proyecto actuará como respaldo del mayor parque eólico terrestre del Reino Unido, Whitelee, cercano a Glasgow, y tendrá una capacidad de almacenamiento de 50 millones de Watios-hora, lo que equivale al consumo medio de 150.000 hogares/hora.
Summarise
El proyecto actuará como respaldo del mayor parque eólico terrestre del Reino Unido, Whitelee, cercano a Glasgow, y tendrá una capacidad de almacenamiento de 50 millones de Watios-hora, lo que equivale al consumo medio de 150.000 hogares/hora.
En momentos de baja demanda de energía de los hogares o de la industria, el parque eólico podrá seguir generando a máxima capacidad, y el excedente de energía limpia generada se almacenará en la batería.
Cuando la demanda sea inferior a la capacidad de g

<a href="http://www.eleconomista.es/energia/noticias/10658607/07/20/Iberdrola-encarga-a-Ingeteam-su-mayor-bateria-de-ion-litio-en-el-mundo.html">Link</a>